# **Scraping SatuSehat (PeduliLindungi) Link News**

Salah satu cara untuk melakukan web scraping adalah dengan menggunakan `Selenium`. Beberapa kelebihan dari Selenium, yaitu:
- Selenium dapat berinteraksi dengan elemen di halaman web, seperti mengklik tombol dan menavigasi antar halaman yang sangat berguna untuk mengakses konten yang dimuat secara dinamis.
- Selenium dapat digunakan pada berbagai browser, salah satunya adalah Google Chrome.




Pada kesempatan ini, kami dari kelompok 2 akan melakukan scraping link berita yang berhubungan dengan aplikasi SatuSehat (PeduliLindungi) menggunakan `Selenium` pada Google Chrome.

# Set Up

Sebelum memulai lebih lanjut, kita harus menyiapkan lingkungan di Ubuntu dengan mengunduh dan menginstal Google Chrome dan ChromeDriver agar Selenium dapat mengotomatisasi browser.

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,447 kB]
Get:14 http://archive.ubu

In [ ]:
# Install Libraries
!pip install chromedriver-autoinstaller
!pip install dateparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
# Import library
import re
import csv
import sys
import time
import dateparser
import pandas as pd
import chromedriver_autoinstaller
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Konfigurasi opsi Chrome
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Agar berjalan di mode tanpa antarmuka
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/129/chromedriver'

# Scraping Link Artikel Berita di Google

In [ ]:
# Inisialisasi list untuk menyimpan semua data
all_articles = set()

In [ ]:
# Fungsi untuk memodifikasi link Google News
def rearrange_news_link(url):
  # Memisahkan URL berdasarkan pola 'start=' diikuti angka dan '&sa='
  url_split = re.split(r"start=(\d+)&sa=", url)
  # Mengembalikan URL yang telah diubah
  return url_split[0] + 'start=', '&sa=' + url_split[-1]

In [ ]:
# Fungsi untuk hanya mengambil bagian utama dari link berita yang didapatkan
def get_google_main_url(url):
  return re.findall(r"^(.*?)(?=&)", url)[0]

In [ ]:
def scrape_google_news_link(result_set, number, url):
    # Memisahkan URL menjadi bagian yang bisa diproses
    url1, url2 = rearrange_news_link(url)

    # Mendapatkan URL utama dari URL yang telah dipisahkan
    main_url = get_google_main_url(url1)
    print(f"Mengakses URL: {main_url}")

    # Menginisialisasi driver Chrome dengan opsi yang sudah ditentukan
    driver = webdriver.Chrome(options=chrome_options)
    article_xpath = "//div[contains(@class, 'MjjYud')]/div"

    try:
        # Mengambil artikel dari URL yang diberikan, sebanyak <number> artikel
        for i in range(0, number, 10):
            # Membuat URL untuk halaman artikel berikutnya
            page = url1 + str(i) + url2
            driver.get(page)
            time.sleep(2)  # Memberi waktu untuk halaman memuat

            # Menemukan elemen yang berisi artikel
            articles = driver.find_element(By.XPATH, article_xpath)
            articles = articles.find_elements(By.CLASS_NAME, "SoaBEf")
            print(f"Mengumpulkan artikel dari {i+1} hingga {i+10}")

            # Mengambil link, judul, dan tanggal dari setiap artikel
            for article in articles:
                # Mengambil link dari elemen <a>
                a = article.find_element(By.TAG_NAME, "a").get_attribute("href")

                 # Mengambil judul berita dari elemen dengan class "n0jPhd ynAwRc MBeuO nDgy9d"
                try:
                    title = article.find_element(By.XPATH, ".//div[@class='n0jPhd ynAwRc MBeuO nDgy9d']").text
                except NoSuchElementException:
                    title = None

                # Mengambil tanggal dari elemen dengan kelas 'OSrXXb'
                try:
                    date_str = article.find_element(By.CLASS_NAME, "OSrXXb").text
                    date = dateparser.parse(date_string=date_str, languages=['en', 'id'])
                    date = date.strftime('%Y-%m-%d') if date else None
                except NoSuchElementException:
                    date = None  # Mengisi dengan None jika tidak ditemukan

                # Mengambil nama portal dari elemen dengan class 'MgUUmf NUnG9d'
                try:
                    portal = article.find_element(By.XPATH, ".//div[@class='MgUUmf NUnG9d']").text
                except NoSuchElementException:
                    portal = None

                # Menambahkan tuple (link, judul, tanggal, portal, hashtag) ke hasil
                result_set.add(tuple([a, title, date, portal]))

    except NoSuchElementException as e:
        # Jika elemen artikel tidak ditemukan
        print('Halaman artikel sudah tidak tersedia')

    finally:
        # Menutup driver setelah proses selesai
        driver.close()
        print(f"Proses selesai. Total link artikel unik yang berhasil diambil: {len(result_set)}")
        print('----------------------------------------------------------')


In [ ]:
# Link Google News yang akan di-scraping
# satu sehat
scrape_google_news_link(all_articles, 1000, 'https://www.google.com/search?q=satu+sehat&sca_esv=449428b1edf18ef7&tbm=nws&sxsrf=ADLYWIIiJ31AXdt_1uPlcuRXCrWbOCJRPg:1725864313300&ei=eZneZv-IEoKgseMPiN7GyQs&start=0&sa=N&ved=2ahUKEwi_qczKobWIAxUCUGwGHQivMbk4ChDy0wN6BAgDEAQ&biw=1536&bih=703&dpr=1.25')
# peduli lindungi
scrape_google_news_link(all_articles, 1000, 'https://www.google.com/search?q=peduli+lindungi&sca_esv=449428b1edf18ef7&tbm=nws&sxsrf=ADLYWII8LRnsJNcGclJCB8Yvnvf7td5KGQ:1725864448748&ei=AJreZra2Ld-R4-EP4YLriAM&start=0&sa=N&ved=2ahUKEwi2tpeLorWIAxXfyDgGHWHBGjE4ChDy0wN6BAgDEAQ&biw=1536&bih=703&dpr=1.25')
# peduli lindungi covid 19
scrape_google_news_link(all_articles, 1000, 'https://www.google.com/search?q=peduli+lindungi+covid+19&sca_esv=449428b1edf18ef7&tbm=nws&sxsrf=ADLYWIJyBmmyIspHbsCsZmAnDpYCI7bcJg:1725864485869&ei=JZreZp3gNLLB4-EP65bUkQc&start=0&sa=N&ved=2ahUKEwjdhvGcorWIAxWy4DgGHWsLNXI4ChDy0wN6BAgEEAQ&biw=1536&bih=703&dpr=1.25')
# layanan peduli lindungi
scrape_google_news_link(all_articles, 1000, 'https://www.google.com/search?q=layanan+peduli+lindungi&sca_esv=449428b1edf18ef7&tbm=nws&sxsrf=ADLYWIK-xWiGNOvjvOTzKmIHXYcHTmeSfA:1725864527340&ei=T5reZoW_FMiN4-EPzbq24Q4&start=0&sa=N&ved=2ahUKEwjFotSworWIAxXIxjgGHU2dLew4ChDy0wN6BAgBEAQ&biw=1536&bih=703&dpr=1.25')

Mengakses URL: https://www.google.com/search?q=satu+sehat
Mengumpulkan artikel dari 1 hingga 10
Mengumpulkan artikel dari 11 hingga 20
Mengumpulkan artikel dari 21 hingga 30
Mengumpulkan artikel dari 31 hingga 40
Mengumpulkan artikel dari 41 hingga 50
Mengumpulkan artikel dari 51 hingga 60
Mengumpulkan artikel dari 61 hingga 70
Mengumpulkan artikel dari 71 hingga 80
Mengumpulkan artikel dari 81 hingga 90
Halaman artikel sudah tidak tersedia
Proses selesai. Total link artikel unik yang berhasil diambil: 84
----------------------------------------------------------
Mengakses URL: https://www.google.com/search?q=peduli+lindungi
Mengumpulkan artikel dari 1 hingga 10
Mengumpulkan artikel dari 11 hingga 20
Mengumpulkan artikel dari 21 hingga 30
Mengumpulkan artikel dari 31 hingga 40
Mengumpulkan artikel dari 41 hingga 50
Mengumpulkan artikel dari 51 hingga 60
Mengumpulkan artikel dari 61 hingga 70
Mengumpulkan artikel dari 71 hingga 80
Mengumpulkan artikel dari 81 hingga 90
Mengumpulkan arti

In [ ]:
# Contoh isi link_set
for i, data in enumerate(all_articles):
  if (i == 5):
    break
  print(data)

('https://www.hulondalo.id/kesehatan/9647774737/aplikasi-peduli-lindungi-tak-akan-dipakai-lagi-ini-penggantinya', 'Aplikasi Peduli Lindungi Tak Akan Dipakai lagi, Ini Penggantinya - Hulondalo', '2023-02-28', 'Hulondalo')
('https://beritajatim.com/semua-pusat-perbelanjaan-kota-kediri-buka-dan-gunakan-aplikasi-peduli-lindungi', 'Semua Pusat Perbelanjaan Kota Kediri Buka dan Gunakan Aplikasi Peduli Lindungi', '2021-10-08', 'beritajatim.com')
('https://www.idntimes.com/news/world/sonya-michaella/aplikasi-pedulilindungi-akan-berubah-jadi-satu-sehat-mobile', 'PeduliLindungi Bakal Berubah Jadi Satu Sehat Mobile', '2023-02-18', 'IDN Times')
('https://dayaknews.com/kapuas/masuk-kantor-diskominfo-kapuas-scan-barcode-peduli-lindungi/', 'MASUK KANTOR DISKOMINFO KAPUAS SCAN BARCODE PEDULI LINDUNGI', '2022-02-11', 'Dayak News')
('https://indonesiabaik.id/index.php/infografis/makin-mudah-checkin-pedulilindungi-via-web', 'Makin Mudah CheckIn PeduliLindungi via WEB', '2022-10-25', 'Indonesia Baik')


## Ubah Set Menjadi DataFrame

In [ ]:
#newsLink_df = pd.DataFrame(data=all_articles, columns=['link', 'tanggal'])
newsLink_df = pd.DataFrame(data=all_articles, columns=['link', 'judul', 'tanggal', 'portal'])

In [ ]:
# Konversi tanggal menjadi datetime, urutkan dari terbaru ke terlama
newsLink_df['tanggal'] = pd.to_datetime(newsLink_df['tanggal'], format='%Y-%m-%d')
newsLink_df = newsLink_df.sort_values(by='tanggal', ascending=False, ignore_index=True)

In [ ]:
newsLink_df

,link,judul,tanggal,portal
0,https://en.antaranews.com/news/328510/health-m...,Health Ministry integrates 119 emergency numbe...,2024-10-06,ANTARA News
1,https://cahayasiang.id/dinilai-peduli-lindungi...,"Dinilai Peduli Lindungi Para Pekerja, BPJS Ket...",2024-09-22,Cahaya Siang
2,https://sippn.menpan.go.id/pelayanan-publik/82...,Sistem Informasi Pelayanan Publik,2024-09-15,SIPPN
3,https://memorandum.disway.id/read/114165/pemer...,Pemerintah Tingkatkan Layanan Haji dan Umrah M...,2024-09-15,Memorandum.co.id
4,https://www.bicaranetwork.com/kesehatan/295134...,Tutorial Cara Isi Formulir Satu Sehat Pasca Pu...,2024-09-06,Bicara Network
...,...,...,...,...
307,https://citizenlab.ca/2020/12/unmasked-ii-an-a...,Unmasked II: An Analysis of Indonesia and the ...,2020-12-21,The Citizen Lab
308,https://sahabatsicepat.com/aplikasi-peduli-lin...,Aplikasi Peduli Lindungi Untuk Tracing Covid-19,2020-04-22,Sahabat SiCepat
309,http://www.thejakartapost.com/life/2020/04/21/...,What to know before using PeduliLindungi surve...,2020-04-21,The Jakarta Post
310,https://www.geospatialworld.net/apps/covid-19/...,Indonesia launches PeduliLindungi app to take ...,2020-04-10,Geospatial World


## Simpan Link Artikel Berita ke CSV dan Excel

In [ ]:
output_file = 'link_berita_satusehat.csv'
newsLink_df.to_csv(output_file, index=False)

message = f"Successfully output the results to the {output_file} file"
print('-' * len(message))
print(message)
print(f"Successfully got {len(all_articles)} unique article links")

---------------------------------------------------------------------
Successfully output the results to the link_berita_satusehat.csv file
Successfully got 312 unique article links


In [ ]:
output_file2 = 'link_berita_satusehat.xlsx'
newsLink_df.to_excel(output_file2, index=False)